In [1]:
# Commit hash e data di questa revisione

! git log --pretty=format:'%h %ai' -n 1

ca28637 2019-06-19 11:12:49 +0200

# Un interprete per il linguaggio Logo

Il progetto richiede l'implementazione di un interprete per il [linguaggio Logo](https://en.wikipedia.org/wiki/Logo_(programming_language)) (così come dettagliato nella sezione seguente). A tal fine:

* deve essere realizzata una **grammatica** derivante dalle specifiche illustrate di seguito (eventualmente prendendo spunto dalle grammatiche [ucb-logo](https://github.com/antlr/grammars-v4/tree/master/ucb-logo), o [logo](https://github.com/antlr/grammars-v4/blob/master/logo));
* deve essere implementato un **parser**, costruito manualmente, o utilizzando ANTLR;
* deve essere implementato un **interprete**, di tipo ricorsivo, o iterativo (basato su *code threading*).

L'interprete può fare uso di una libreria a *runtime* per la parte grafica come:

* il pacchetto [turtle](https://docs.python.org/3.3/library/turtle.html?highlight=turtle) delle API standard di Python,
* il pacchetto [tartaruga](https://github.com/mapio/tartaruga) o analogo per Java,
* il codice in [papert](https://github.com/rmmh/papert) o analogo per JavaScipt.

A seconda delle scelte (circa le modalità di implementazione del parser e dell'interprete), è possibile concordare con il docente una riduzione della porzione di specifiche del linguaggio Logo che l'interprete deve implementare.

## Modalità di sviluppo e consegna

Il codice deve essere consegnato tramite un **apposito repository** su GitHub creato dal docente su richiesta degli studenti. Gli studenti sono incoraggiati ad effettuare frequenti **commit** periodici per testimoniare il processo di sviluppo e sono altresì incoraggiati ad effettuare (anche con una frequenza ridotta rispetto ai *commit*) dei **push** periodici al fine di informare il docente dello stato di avanzamento del progetto (cosa che rende possibili, anche se non garantite, delle eventuali *code review* del codice da parte del docente). Al termine del lavoro, con congruo anticipo rispetto alla data del colloquio, lo studente effettua un commit con messaggio `CONSEGNA FINALE` che segna il termine del processo di sviluppo e l'inizio della valutazione da parte del docente. Chi non è in grado di usare `git` e GitHub può documentarsi per tempo in rete su come usare tali strumenti.

Non è richiesta la stesura di una *relazione*; per questa ragione è indicato che il codice contenga un **minimo di documentazione** atta a guidare il docente nella comprensione della sua architettura, delle scelte progettuali ed implementative e del comportamento delle sue varie parti. Si consiglia di redigere tale documentazione sotto forma di *commenti nel codice*, usando eventualmente (solo se si è in grado e senza perdere tempo) uno degli strumenti di documentazione automatica disponibili per il linguaggio scelto.

Il codice deve consentire un minimo di **testing**, se non altro sotto forma di test di accettazione sviluppati a partire dai sorgenti Logo forniti in calce a questa documentazione. <mark>Siete liberi di implementare i vostri test, o di usare il file [tests.py](tests.py) messo a disposizione del docente.</mark>

## Specifiche del linguaggio Logo

Le specifiche del linguaggio Logo a cui fare riferimento nello svolgimento del progetto sono basate sul [Berkeley Logo User Manual](https://people.eecs.berkeley.edu/~bh/usermanual) [BLUM]; a ciascuna *sezione* di tale manuale (indicata in esso da una sottolineatura composta di segni `=`) corrisponde qui una sezione che chiarisce in che termini essa debba essere rispettata dall'interprete.


### GETTER/SETTER VARIABLE SYNTAX

Le considerazioni di questa sezione del BLUM sono sostanzialmente da ignorare.

Si assuma viceversa che l'interprete abbia un **unico spazio dei nomi** per *variabili* e *procedure* (non primitive, ossia definite dall'utente). <mark>Unica eccezione a questa regola sono i parametri formali (input) delle procedure definite dall'utente che *ombreggiano* le eventuali variabili dallo stesso nome (altrimenti, ad esempio, non sarebbe possibile avere procedure ricorsive).</mark>

#### I riferimenti a variabile

Nel comando `MAKE` il primo input è un riferimento a variabile. L'interprete può limitarsi ai casi ni cui tale riferimento sia dato da `"name` (ossia la variabile da assegnare è `name`), oppure  `THING varname`, o `:varname` (ossia la variabile da assegnare è il valore della variabile `varname`).

#### Le invocazioni di procedure definite dall'utente

Per quanto concerne l'*invocazione delle procedure*, usualmente in Logo  essa avviene indicando il nome della procedura seguito dalle espressioni corrispondenti ai suoi *input* (o argomenti). Dato che le procedure *primitive* (ossia che sono parte del linguaggio, o predefinite) hanno un numero di input noto a priori, non pongono problemi di parsing. 

Qquesto non è vero per le procedure definite dall'utente (come si vedrà, tramite la procedura di "forma speciale" `TO`). 

Se `F` e `G` sono due procedure il cui numero di input non è noto, l'espressione `F G 1 2 3` può essere intesa come `F (G 1 2 3)`, o `F (G 1 2) 3`, oppure `F (G 1) 2 3`.

Per risolvere questo problema in modo semplice, l'interprete assume che (diversamente dal Logo descritto nel BLUM), **l'invocazione di una procedura definita dall'utente che ha un numero di argomenti diverso da 1 è racchiusa tra parentesi** (un po' come avviene per le procedure `SUM` e `PRODUCT` quando il numero di input è maggiore di 2). 

Con tale restrizione, l'espressione `F G 1 2 3` diventa illegittima (non è possibile che sia `F` che `G` siano unarie), le tre interpretazioni possibili elencate sopra devono essere indicate come `F (G 1 2 3)`, o `(F (G 1 2) 3)`, oppure `(F G 1 2 3)` (a seconda che, rispettivamente, `F` sia unaria, siano entrambe binarie, o `G` sia unaria).


### ENTERING AND LEAVING LOGO

Questa sezione del BLUM è da ignorare. L'interprete deve essere implementato come una funzione che prende in ingresso una stringa corrispondente al codice Logo e non restituisce alcun valore (ma produce l'eventuale disegno descritto nel codice).


### TOKENIZATION

Le considerazioni descritte in questa sezione del BLUM sono da ignorare in favore di una versione molto più semplificata del processo di divisione in *token* descritta di seguito. 

Assumete che gli *identificatori* (nomi di *procedure* e *variabili*) siano composti da una sequenza non vuota di lettere alfabetiche, che i *numeri* (interi e reali) abbiano l'usuale struttura lessicale (vedi gli esempi nei sorgenti) e che le *stringhe* siano costituite dal carattere `"` seguito da uno o più caratteri alfanumerici (spazio escluso). Sono da considerarsi riservati (oltre ai nomi di tutte le procedure primitive) gli identificatori `END` (usato come terminatore della "forma speciale" per la procedura `TO` di cui si parla più avanti) e la coppia`TRUE` e `FALSE` che corrisponde ai letterali per i tipi *booleani* vero e falso; a tal proposito si ricorda che le stringhe `"true` e `"false` (a meno del *case*) coincidono quanto a valore di verità con i precedenti letterali.

Assumete che i *token* siano separati dall'usuale *whitespace* (spazi, tabulazioni e "a capo") e che quindi più invocazioni di procedura possano susseguirsi sulla stessa linea; analogamente, nella forma `TO` non è necessario che intercorra un "a capo" tra l'ultimo input e la prima procedura del suo corpo.


### DATA STRUCTURE PRIMITIVES

Questa sezione del BLUM è da ignorare; oltre a numeri, stringhe <mark>e *boolean* (che sono gli unici consentiti come input delle operazioni logiche (connettivi)</mark>, l'unica struttura non elementare che l'interprete deve gestire sono le liste letterali <mark>di istruzioni</mark> (ossia presenti nel sorgente e delimitate da `[` e `]`), richieste come tipo di input da alcune procedure.


### COMMUNICATION 

Di questa sezione del BLUM l'interprete deve implementare solo la procedura `PRINT` per quanto concerne l'output.  Per quanto concerne l'input, deve essere implementata una variante della procedura `READWORD` (`RW`) tale per cui se il valore letto dal flusso di ingresso standard (fino al primo *whitespace* escluso) può essere convertito in numero, essa restituisca un numero, viceversa resituisca la stringa corrispondente ai caratteri del flusso di ingresso standard (fino al primo *whitespace* escluso).


### ARITHMETIC

Di questa sezione del BLUM l'interprete deve implementare tutte le procedure elencate in `NUMERIC OPERATIONS` tranne `ISEQ` e `RSEQ` (fate attenzione alla precedenza del `-`/`MINUS` unario), tutti i predicati in `PREDICATES` <mark>(che ammettono solo input di tipo numerico)</mark> e le procedure `RANDOM NUMBERS`. Tralasciate le specifiche che riguardano comportamenti delle funzioni trascendenti e della radice che non siano conformi ai comportamenti offerti da tali funzioni nell'ambiente d'esecuzione dell'interprete.

L'interprete deve inoltre implementare le versioni infisse delle procedure che le prevedono, prestando particolare attenzione alla loro precedenza e all'associatività.


### LOGICAL OPERATIONS

Di questa sezione del BLUM l'interprete deve implementare tutte le procedure che rappresentano i connettivi <mark><del>e le loro versioni infisse</del></mark> (prestando particolare attenzione alla precedenza, sopratutto in rapporto alle operazioni aritmetiche infisse). Va inoltre rispettata la [short-circuit evaluation](https://en.wikipedia.org/wiki/Short-circuit_evaluation). <mark>Osservate che il linguaggio non prevede alcuna conversione implicita da stringhe e numeri in *boolean*, pertanto i connettivi devono avere come input esclusivamente espressioni di tipo *boolean* (derivanti dalle operazioni della sezione `PREDICATES`).</mark>


### GRAPHICS

Di questa sezione del BLUM l'interprete deve implementare tutte le procedure di `TURTLE MOTION` e di `TURTLE AND WINDOW CONTROL` fino a `CLEARSCREEN`/`CS` (inclusa). Tra le procedure di `PEN AND BACKGROUND CONTROL` deve implementare solo `PENDOWN`/`PD`, `PENUP`/`PU`, `SETPENCOLOR`/`SETPC` (solo nella forma in cui l'input è una lista di valori RGB) e `SETPENSIZE`.


### WORKSPACE MANAGEMENT

Di questa sezione del BLUM l'interprete deve implementare, per la parte di `PROCEDURE DEFINITION`, solo la procedura `TO` (e `END`) con i soli *required input* (ma senza *optional*/*rest* input o numero di input di *default*).  Per la parte di `VARIABLE DEFINITION` solo `MAKE` e `THING`/`:`.


### CONTROL STRUCTURES

Di questa sezione del BLUM l'interprete deve implementare le sole strutture di controllo `REPEAT`, `IF` (nella forma con un solo input), `IFELSE`, `STOP` (si osservi che questa struttura di controllo equivale all'usuale `return` senza espressione, non a un `break`), `OUTPUT`/`OP` e `WHILE`. <mark>Si osservi che l'istruzione `IFELSE` può essere usata come una sorta di *operatore ternario*, tale cioè da costituire una espressione coincidente (a seconda della condizione) con il valore di una delle sue due liste in input; in tal senso, il *valore* di una lista è ottenuto valutando da sinistra a destra le istruzioni che contiene ed è pari al valore dell'ultima espressione valutata. Ad esempio `MAKE "A 0 IFELSE "false [MAKE "A 1] [MAKE "B 2 :B MAKE "C 3]` ha per valore `2` (infatti l'ultima espressione valutata è `:B` che vale `2` per via del `MAKE` valutato prima di essa; inoltre la variabile di nome `A` vale `0` perché ovviamente la prima lista non viene valutata.</mark>


### MACROS, ERROR PROCESSING, SPECIAL VARIABLES, INTERNATIONALIZATION

Queste sezioni del BLUM sono da ignorare; riguardo alla gestione degli errori, l'interprete deve quanto meno catturare gli errori di esecuzione del linguaggio in cui è implementato e segnalare gli errori del sorgente Logo o della sua esecuzione in modo il più possibile chiaro (non necessariamente indicando il luogo del sorgente a cui è dovuto l'errore).


## Elenco di primitive (e keyword)

In conclusione, l'elenco di primitive da implementare è il sequente

```
AND, ARC, ARCTAN, BACK, CLEAN, CLEARSCREEN, COS, DIFFERENCE, END,
EQUALP, EXP, FALSE, FORWARD, GREATEREQUALP, GREATERP, HIDETURTLE, HOME,
IF, IFELSE, INT, LEFT, LESSEQUALP, LESSP, LN, LOG10, MAKE, MODULO, NOT,
NOTEQUALP, OR, OUTPUT, PENDOWN, PENUP, POWER, PRINT, PRODUCT, QUOTIENT,
RADARCTAN, RADCOS, RADSIN, RANDOM, READWORD, REMAINDER, REPEAT, RERANDOM,
RIGHT, ROUND, SETHEADING, SETPENCOLOR, SETPENSIZE, SETX, SETXY, SETY,
SHOWTURTLE, SIN, SQRT, STOP, SUM, THING, TO, TRUE, WHILE
```

## <mark>Testing</mark>


Il file [tests.py](tests.py) definisce due dizionari `PARSER_TESTS` e `INTERPRETER_TESTS` che contengono, rispettivamente, alcuni test per validare il comportamento del parser e dell'interprete. Tale file contiene anche del codice per svolgere in modo automatico dei test basati su tali dizionari (e delle istruzioni di funzionamento). 

Tali test saranno utilizzati dal docente durante l'orale per valutare il comportamento dell'implementazione presentata.

Per quanto riguarda il test relativi alla parte grafica, verrà usato `PARSERS_TESTS['koch']` (corrispondente al il [fiocco di neve di Koch](https://en.wikipedia.org/wiki/Koch_snowflake)) più eventuali altri test analoghi.

## Note

Chi intende usare il modulo [turtle](https://docs.python.org/3.3/library/turtle.html?highlight=turtle) delle API di Python all'interno di Jupyter, deve concludere ogni cella in cui i suoi metodi sono invocati con la porzione di codice seguente

```python
turtle.done()
try:
    turtle.bye()
except turtle.Terminator:
    pass
```

e chiudere ogni la finestra grafica che si apre all'esecuzione della cella, al fine di evitare potenziali crash. Ad esempio, il codice seguente disegna una stella e termina senza crash.

In [2]:
import turtle

for _ in range(5):
    turtle.forward(200)
    turtle.right(144)

turtle.done()
try:
    turtle.bye()
except turtle.Terminator:
    pass